## 1. Theory: OLS with Newey–West (HAC) standard errors

In the paper, the spillover index (or its log-return) is regressed on a set of exogenous variables (fuel prices, loads, RES shares, …) using OLS and Newey–West robust standard errors to correct for heteroskedasticity and autocorrelation:

$$
S_t = \beta_0 + \beta_1 x_{1,t} + \dots + \beta_K x_{K,t} + \varepsilon_t,
$$

where

* ($S_t$) is the endogenous variable (e.g. total spillover, or “to others”, “from others”, etc.),
* ($x_{k,t}$) are exogenous regressors (here: CO₂/EUA, coal, TTF, …),
* ($\varepsilon_t$) is an error term that may be serially correlated and heteroskedastic.

OLS still gives unbiased/consistent point estimates under standard conditions, but the usual covariance estimator ($ \hat{\sigma}^2(X'X)^{-1}$) is wrong when errors are autocorrelated or heteroskedastic. Newey–West (HAC) replaces it with

$$
\widehat{\mathrm{Var}}*{\text{NW}}(\hat\beta)
= (X'X)^{-1} \left( \sum*{|\ell|\le L} w_\ell \Gamma_\ell \right) (X'X)^{-1},
$$

where

* ($\Gamma_\ell = \frac{1}{T} \sum_{t=\ell+1}^T u_t u_{t-\ell}' X_t' X_{t-\ell}$) are lag-($\ell$) autocovariances of residuals,
* ($w_\ell$) are Bartlett weights,
* ($L$) is the chosen truncation lag (“maxlags” in `statsmodels`).

In practice we:

1. Estimate OLS: ($\hat\beta = (X'X)^{-1}X'Y$).
2. Compute Newey–West robust covariance with chosen lag ($L$).
3. Use this covariance for t-stats, p-values and confidence intervals.

The code below implements exactly that via `statsmodels` (`cov_type="HAC"` and `cov_kwds={"maxlags": L}`), and runs *one regression per endogenous series*.





In [17]:
# %% 
# Imports and paths

from pathlib import Path
from typing import Dict, List, Optional, Tuple
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# Paths
OUT_DIR = Path("../data/endogenous/prices/differenced")
endog_path = OUT_DIR / "endogenous_tsi_diff.parquet"
exog_path = Path("../data/exogenous/all_exog_ready.parquet")


In [10]:
# %%
# Helper to load parquet and convert to wide format

def load_parquet_to_wide(
    path: Path,
    date_col: str = "date",
    series_col: str = "series",
    value_col: str = "value",
) -> pd.DataFrame:
    """
    Load a parquet file and return a wide DataFrame with a DatetimeIndex.

    If the file is in long format [date, series, value, ...], pivot to wide.
    If it's already wide with a 'date' column, just set 'date' as index.
    """
    df = pd.read_parquet(path)

    if {date_col, series_col, value_col}.issubset(df.columns):
        # Long format -> pivot to wide
        df[date_col] = pd.to_datetime(df[date_col])
        df_wide = (
            df.pivot(index=date_col, columns=series_col, values=value_col)
              .sort_index()
        )
    else:
        # Assume already wide with 'date' column
        if date_col not in df.columns:
            raise ValueError(
                f"'date' column not found in {path}. "
                "Add it or adapt `date_col` in load_parquet_to_wide()."
            )
        df = df.copy()
        df[date_col] = pd.to_datetime(df[date_col])
        df_wide = df.set_index(date_col).sort_index()

    df_wide.index = pd.to_datetime(df_wide.index)
    return df_wide


In [11]:
# %%
# Prepare regression data: align endogenous/exogenous and drop NAs

def prepare_regression_data(
    endog_path: Path,
    exog_path: Path,
    exog_cols: Optional[List[str]] = None,
) -> Tuple[pd.DataFrame, List[str], List[str]]:
    endog_wide = load_parquet_to_wide(endog_path)
    exog_wide = load_parquet_to_wide(exog_path)

    data = endog_wide.join(exog_wide, how="inner")

    endog_names = list(endog_wide.columns)

    if exog_cols is None:
        exog_names = list(exog_wide.columns)
    else:
        missing = [c for c in exog_cols if c not in exog_wide.columns]
        if missing:
            raise ValueError(f"Missing exogenous columns in exog data: {missing}")
        exog_names = exog_cols

    data = data[endog_names + exog_names].dropna()
    return data, endog_names, exog_names


In [12]:
# %%
# Newey–West regressions + coefficient table

def run_nw_regressions(
    data: pd.DataFrame,
    endog_names: List[str],
    exog_names: List[str],
    maxlags: int = 5,
) -> Dict[str, sm.regression.linear_model.RegressionResultsWrapper]:
    results = {}
    X = sm.add_constant(data[exog_names])

    for y_col in endog_names:
        y = data[y_col]
        model = sm.OLS(y, X)
        res = model.fit(cov_type="HAC", cov_kwds={"maxlags": maxlags})
        results[y_col] = res

    return results


In [13]:
def build_coef_table(
    results: Dict[str, sm.regression.linear_model.RegressionResultsWrapper],
    exog_names: List[str],
) -> pd.DataFrame:
    rows = []
    for y_name, res in results.items():
        for var in ["const"] + exog_names:
            rows.append(
                {
                    "endog": y_name,
                    "variable": var,
                    "coef": res.params.get(var, np.nan),
                    "std_err": res.bse.get(var, np.nan),
                    "t": res.tvalues.get(var, np.nan),
                    "pval": res.pvalues.get(var, np.nan),
                    "ci_low": res.conf_int().loc[var, 0] if var in res.params.index else np.nan,
                    "ci_high": res.conf_int().loc[var, 1] if var in res.params.index else np.nan,
                }
            )
    return pd.DataFrame(rows)

In [14]:
def build_model_table(
    results: Dict[str, sm.regression.linear_model.RegressionResultsWrapper],
    maxlags: int,
) -> pd.DataFrame:
    rows = []
    for y_name, res in results.items():
        rows.append(
            {
                "endog": y_name,
                "nobs": int(res.nobs),
                "r2": res.rsquared,
                "adj_r2": res.rsquared_adj,
                "f_stat": getattr(res, "fvalue", np.nan),
                "f_pval": getattr(res, "f_pvalue", np.nan),
                "aic": res.aic,
                "bic": res.bic,
                "dw": sm.stats.stattools.durbin_watson(res.resid),
                "nw_maxlags": maxlags,
            }
        )
    return pd.DataFrame(rows).sort_values("endog").reset_index(drop=True)


def save_text_summaries(
    results: Dict[str, sm.regression.linear_model.RegressionResultsWrapper],
    out_dir: Path,
    filename: str = "nw_model_summaries.txt",
):
    out_dir.mkdir(parents=True, exist_ok=True)
    fp = out_dir / filename
    with fp.open("w", encoding="utf-8") as f:
        for y_name, res in results.items():
            f.write("=" * 90 + "\n")
            f.write(f"ENDOG: {y_name}\n")
            f.write("=" * 90 + "\n\n")
            f.write(res.summary().as_text())  # classic statsmodels summary table
            f.write("\n\n")
    print(f"Saved model summaries to: {fp}")

In [15]:
# %%
# Plot helper for one endogenous variable

def plot_coefficients_for_endog(coef_df: pd.DataFrame, endog_name: str):
    subset = coef_df[(coef_df["endog"] == endog_name) & (coef_df["variable"] != "const")].copy()
    z = 1.96
    fig, ax = plt.subplots(figsize=(9, 4))
    ax.errorbar(
        x=subset["variable"],
        y=subset["coef"],
        yerr=z * subset["std_err"],
        fmt="o",
        capsize=4,
    )
    ax.axhline(0.0, linestyle="--", linewidth=1)
    ax.set_title(f"Newey–West coefficients for {endog_name}")
    ax.set_ylabel("Coefficient")
    ax.set_xlabel("Exogenous variable")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [6]:
# %%
# Main workflow cell

# Choose which exogenous variables to use:
#   - Set exog_cols = None to use ALL exogenous columns from the parquet.
#   - Or specify a list, e.g. exog_cols = ["d_eua", "d_coal", "d_ttf"]
exog_cols = None

# Load and prepare data
data, endog_names, exog_names = prepare_regression_data(
    endog_path=endog_path,
    exog_path=exog_path,
    exog_cols=exog_cols,
)

print("Endogenous series:", endog_names)
print("Exogenous variables used:", exog_names)
print("Final sample size:", len(data))


Endogenous series: ['d_tsi_mhar_recov_neg', 'd_tsi_mhar_recov_pos', 'd_tsi_mhar_recov', 'd_tsi_mhar_revar']
Exogenous variables used: ['load_energy_mwh_pt', 'flow_net_mw_ES_PT', 'd_TTF', 'd_co2', 'd_coal', 'd_load_energy_mwh_es', 'd_load_energy_mwh_fr', 'd_flow_net_mw_ES_FR', 'd_cac_eur_pts', 'd_ibex_eur_pts', 'd_psi_eur_pts', 'iberian_exception']
Final sample size: 1076


In [18]:

# exog_cols = None  # or list of columns

data, endog_names, exog_names = prepare_regression_data(
    endog_path=endog_path,
    exog_path=exog_path,
    exog_cols=exog_cols,
)

print("Endogenous series:", endog_names)
print("Exogenous variables used:", exog_names)
print("Final sample size:", len(data))

nw_lags = 10

results = run_nw_regressions(
    data=data,
    endog_names=endog_names,
    exog_names=exog_names,
    maxlags=nw_lags,
)

# --- Classical summary output (printed in notebook) -------------------------
for y_name in endog_names:
    print("\n" + "#" * 110)
    print(f"OLS + Newey–West (HAC) — ENDOG = {y_name}  |  maxlags={nw_lags}")
    print("#" * 110)
    print(results[y_name].summary())

# --- Save summaries to a text file -----------------------------------------
save_text_summaries(results, OUT_DIR, filename="nw_model_summaries.txt")

# --- Save tidy outputs (CSV) ------------------------------------------------
coef_df = build_coef_table(results, exog_names)
model_df = build_model_table(results, nw_lags)

RESULTS_DIR = Path("../data/results")
coef_path = RESULTS_DIR / "nw_regression_coefficients.csv"
model_path = RESULTS_DIR / "nw_regression_model_stats.csv"

coef_df.to_csv(coef_path, index=False)
model_df.to_csv(model_path, index=False)

print(f"\nSaved coefficient table to: {coef_path}")
print(f"Saved model stats table to: {model_path}")

display(model_df.head())
display(coef_df.head(20))

Endogenous series: ['d_tsi_mhar_recov_neg', 'd_tsi_mhar_recov_pos', 'd_tsi_mhar_recov', 'd_tsi_mhar_revar']
Exogenous variables used: ['load_energy_mwh_pt', 'flow_net_mw_ES_PT', 'd_TTF', 'd_co2', 'd_coal', 'd_load_energy_mwh_es', 'd_load_energy_mwh_fr', 'd_flow_net_mw_ES_FR', 'd_cac_eur_pts', 'd_ibex_eur_pts', 'd_psi_eur_pts', 'iberian_exception']
Final sample size: 1076

##############################################################################################################
OLS + Newey–West (HAC) — ENDOG = d_tsi_mhar_recov_neg  |  maxlags=10
##############################################################################################################
                             OLS Regression Results                             
Dep. Variable:     d_tsi_mhar_recov_neg   R-squared:                       0.031
Model:                              OLS   Adj. R-squared:                  0.020
Method:                   Least Squares   F-statistic:                     2.943
Date:     

,endog,nobs,r2,adj_r2,f_stat,f_pval,aic,bic,dw,nw_maxlags
0,d_tsi_mhar_recov,1076,0.032479,0.021557,2.030090,0.019109,-720.054542,-655.301468,1.727792,10
1,d_tsi_mhar_recov_neg,1076,0.030550,0.019606,2.943437,0.000484,-338.393907,-273.640833,1.742197,10
2,d_tsi_mhar_recov_pos,1076,0.032015,0.021088,2.769297,0.001012,-692.677902,-627.924828,1.817574,10
3,d_tsi_mhar_revar,1076,0.015280,0.004163,1.857322,0.035756,-506.073362,-441.320287,1.790090,10


,endog,variable,coef,std_err,t,pval,ci_low,ci_high
0,d_tsi_mhar_recov_neg,const,1.975155e-01,7.933349e-02,2.489686,0.012786,4.202470e-02,3.530063e-01
1,d_tsi_mhar_recov_neg,load_energy_mwh_pt,-1.139780e-06,5.315179e-07,-2.144388,0.032002,-2.181537e-06,-9.802445e-08
2,d_tsi_mhar_recov_neg,flow_net_mw_ES_PT,-2.049685e-05,8.546873e-06,-2.398169,0.016477,-3.724841e-05,-3.745282e-06
3,d_tsi_mhar_recov_neg,d_TTF,-2.824907e-03,1.119932e-03,-2.522392,0.011656,-5.019933e-03,-6.298808e-04
4,d_tsi_mhar_recov_neg,d_co2,-1.316377e-03,1.218289e-03,-1.080513,0.279914,-3.704181e-03,1.071426e-03
5,d_tsi_mhar_recov_neg,d_coal,-1.871058e-04,1.202344e-03,-0.155617,0.876335,-2.543658e-03,2.169446e-03
6,d_tsi_mhar_recov_neg,d_load_energy_mwh_es,3.624229e-08,2.908760e-08,1.245970,0.212775,-2.076836e-08,9.325295e-08
7,d_tsi_mhar_recov_neg,d_load_energy_mwh_fr,-1.803707e-08,2.774562e-08,-0.650087,0.515636,-7.241748e-08,3.634335e-08
8,d_tsi_mhar_recov_neg,d_flow_net_mw_ES_FR,-2.602386e-06,5.754323e-06,-0.452249,0.651090,-1.388065e-05,8.675879e-06
9,d_tsi_mhar_recov_neg,d_cac_eur_pts,-8.272939e-05,1.243511e-04,-0.665289,0.505866,-3.264530e-04,1.609942e-04
